In [1]:
import nltk
from nltk.corpus import brown
import pickle
import os
import nltk
import string
from nltk.corpus import brown
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import Word2Vec
import os
import tarfile
import urllib.request
import glob
from lxml import etree
from nltk.tokenize import sent_tokenize, word_tokenize
import pickle
import functools
import operator
import os
import nltk
import string
from nltk.corpus import brown
from nltk.corpus import stopwords
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tqdm import tqdm
import absl.logging

import io
import re
import string
import tqdm

import numpy as np
from tensorflow.keras import layers

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

def save_tokenized_sentences(tokenized_sentences, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(tokenized_sentences, file)

def load_tokenized_sentences(file_path):
    with open(file_path, 'rb') as file:
        tokenized_sentences = pickle.load(file)
    return tokenized_sentences

def preprocess_text_unpacked(sentences):
    table = str.maketrans("", "", string.punctuation) # removing punctuation - it has to be a conscious decision
    tokenized_sentences = [[word.lower().translate(table) for word in sentence] for sentence in sentences]
    stop_words = set(stopwords.words("polish.txt"))
    tokenized_sentences = [[word for word in sentence if word and word not in stop_words] for sentence in tokenized_sentences]
    # tokenized_sentences = functools.reduce(operator.iconcat, tokenized_sentences, [])
    return tokenized_sentences

def mapping(tokens):
    word_to_id = {}
    id_to_word = {}

    for i, token in enumerate(set(tokens)):
        word_to_id[token] = i
        id_to_word[i] = token

    return word_to_id, id_to_word

def generate_training_data(tokens, word_to_id, window_size):
    N = len(tokens)
    X, Y = [], []

    for i in range(N):
        nbr_inds = list(range(max(0, i - window_size), i)) + \
                   list(range(i + 1, min(N, i + window_size + 1)))
        for j in nbr_inds:
            X.append(word_to_id[tokens[i]])
            Y.append(word_to_id[tokens[j]])

    X = np.array(X)
    # X = np.expand_dims(X, axis=0)
    Y = np.array(Y)
    # Y = np.expand_dims(Y, axis=0)

    return X, Y

def batcherize(x_train, y_train, n_batches, n_batch):

    vocab_size = np.max(x_train) + 1
    dataset_length = x_train.shape[0]
    batch_size = int(np.floor(dataset_length / n_batches))

    y_train_batch = np.zeros((batch_size, vocab_size))
    y_train_batch[np.arange(batch_size), y_train[batch_size * n_batch : (n_batch + 1) * batch_size]] = 1

    x_train_batch = np.zeros((batch_size, vocab_size))
    x_train_batch[np.arange(batch_size), x_train[batch_size * n_batch : (n_batch + 1) * batch_size]] = 1

    return x_train_batch, y_train_batch

In [2]:
file_path = './polish_corpus/polish_tokenized_sentences.pkl'
polish_tokenized_sentences = load_tokenized_sentences(file_path)

preprocessed = preprocess_text_unpacked(polish_tokenized_sentences)
# words, ids = mapping(preprocessed)
preprocessed[:5]

[['zatrzasnął',
  'drzwi',
  'mieszkania',
  'razy',
  'przekręcił',
  'klucz',
  'nacisnął',
  'klamkę',
  'sprawdzić',
  'zamknięte',
  'zbiegł',
  'schodach',
  'minął',
  'furtkę',
  'zamknął',
  'znalazł',
  'wąskiej',
  'uliczce',
  'ogródkami',
  'drzemały',
  'majowym',
  'słońcu',
  'trójkątne',
  'ciemnozielone',
  'świerki',
  'jakich',
  'pobliżu',
  'domu'],
 ['bohaterem', 'powieści', 'paźniewskiego', 'miasto', 'krzemieniec'],
 ['czasów', 'słowackiego', 'funkcjonuje', 'liceum', 'płynie', 'ikwa'],
 ['krzemieniec',
  'powieściowy',
  'tamtym',
  'krzemieńcem',
  'miastem',
  'wywołanym',
  'osobistej',
  'pamięci',
  'paźniewskiego'],
 ['swoją', 'drogę', 'miasta', 'autor', 'krótkich', 'dni', 'zaczął', 'daleka']]

In [3]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm


class myWord2Vec:
    def __init__(self, sentences, window, epochs, embedding_dim = 5, negative_samples = 0, unique = False) -> None:
        self.sentences = sentences
        self.window = window
        self.epochs = epochs
        self.negative_samples = negative_samples
        self._create_data()
        self.model = self._create_model(embedding_dim)
        self.unique = unique


    def _create_data(self):
        lst = []
        for x in self.sentences:
            lst += x

        unique_words = list(set(lst))
        self.word_to_id = {}
        self.id_to_word  = {}
        for i, word in enumerate(unique_words):
            self.word_to_id[word] = i
            self.id_to_word[i] = word

        self.vocab_size = len(self.word_to_id)


    def _create_model(self, embedding_dim):
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(self.vocab_size, embedding_dim, embeddings_initializer="RandomNormal", input_shape=(2,)),
            tf.keras.layers.GlobalAveragePooling1D(),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()])
        return model


    def _my_skipgrams(self, sequence, vocabulary_size):
        couples = []
        labels = []
        for i in range(len(sequence)):
            wi = sequence[i]
            window_start = max(0, i - self.window)
            window_end = min(len(sequence), i + self.window + 1)
            for j in range(window_start, window_end):
                if j != i:
                    wj = sequence[j]
                    couples.append([wi, wj])
                    labels.append(1)

        if self.negative_samples > 0:
            num_negative_samples = len(labels) * self.negative_samples
            words = [couple[0] for couple in couples]
            random.shuffle(words)
            for i in range(num_negative_samples):
                word = words[i % len(words)]
                couples.append([word, self.id_to_word[random.randint(1, vocabulary_size - 1)]])
                labels.append(0)

        return  couples, labels


    def _to_one_hot(self, word_index, vocab_size):
        one_hot = np.zeros(vocab_size)
        one_hot[word_index] = 1
        return one_hot


    def train(self):
        y = []
        posTrainSet  = []
        negTrainSet  = []
        for sentence in tqdm(self.sentences):
            if len(sentence) < 2:
                continue

            couples, labels = self._my_skipgrams(sentence, len(sentence))
            for j in range(len(couples)):
                couple = couples[j]
                if labels[j]:
                    posTrainSet.append([self.word_to_id[couple[0]], self.word_to_id[couple[1]]])
                    y.append(labels[j])

                else:
                    negTrainSet.append([self.word_to_id[couple[0]], self.word_to_id[couple[1]]])
                    y.append(labels[j])

        X = np.concatenate([np.array(posTrainSet), np.array(negTrainSet)], axis=0)
        y = np.array(y)
        if self.unique:
            X = np.unique(X, axis = 0)

        self.model.fit(X, y, batch_size=int(X.shape[0]*0.01), epochs=self.epochs)


    def find_similar_words(self, input_word, n):
        embedding_layer = self.model.layers[0]
        word_embeddings = embedding_layer.get_weights()[0]
        input_index = self.word_to_id[input_word]
        input_embedding = word_embeddings[input_index]
        similarities = np.dot(word_embeddings, input_embedding)
        most_similar_indices = similarities.argsort()[::-1][1:n+1]
        most_similar_words = [list(self.word_to_id.keys())[i] for i in most_similar_indices]
        return most_similar_words


model = myWord2Vec(sentences=preprocessed, window=3, epochs=5, embedding_dim=250, negative_samples=5)
model.train()

100%|██████████| 88760/88760 [00:35<00:00, 2482.09it/s]


Epoch 1/5
101/101 [==============================] - 69s 659ms/step - loss: 0.5253 - accuracy: 0.0000e+00
Epoch 2/5
101/101 [==============================] - 67s 667ms/step - loss: 0.4550 - accuracy: 0.0000e+00
Epoch 3/5
101/101 [==============================] - 66s 652ms/step - loss: 0.4481 - accuracy: 0.0000e+00
Epoch 4/5
101/101 [==============================] - 67s 662ms/step - loss: 0.4433 - accuracy: 0.0000e+00
Epoch 5/5
101/101 [==============================] - 66s 657ms/step - loss: 0.4402 - accuracy: 0.0000e+00


In [8]:
preprocessed[150]

['puścili',
 'dwóch',
 'rogatki',
 'trafili',
 'zagajnik',
 'śniegiem',
 'zawalony',
 'jęli',
 'otrząsać',
 'biały',
 'całun']

In [9]:
model.find_similar_words('całun', 5)

['przemycany', 'chaszczach', 'zagoszczą', '394', 'krzywoprzysięstwo']